# Project 3 for CSE142
## CIFAR-10 Classification

Importing necessary packages:

- tensorflow/keras: API to build and train models
- matplotlib: generate graphs
- numpy: perform array operations
- pandas: to work with dataframes
- pickle: for serializing and de-serializing a Python object
- os: for file paths

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import matplotlib.image as mpimg 
import numpy as np
import pandas as pd
# import pickle
# import os, os.path
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.optimizers import Adam

### *Initialize relevant variables*

In [ ]:
labels_df = pd.read_csv("Semi1_Labels.csv") #change this for different versions
train_path = 'Train_Image' 
test_path = 'Test_Image' 
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
#labels_df

## Defining some functions

In [ ]:
def get_image(img_number):
    img = mpimg.imread(train_path + f"/{img_number}.png") 
    return img
def get_test_image(img_number):
    img = mpimg.imread(test_path + f"/{img_number}.png") 
    return img
def get_label(arr):
    max_index_col = np.argmax(arr, axis=0)
    return labels[max_index_col]

In [ ]:
# plt.imshow(get_image(7777))

In [ ]:
# plt.imshow(get_test_image(999))

**Function that prints image info**

In [ ]:
def get_img_info(img):
    rows,cols,channels = img.shape 
    img_size = rows*cols*channels # channels means rgb colors i think, so should always be 3?
    img_to_1D = img.reshape(img_size) #numPy reshape
    print("rows, cols, channels:", rows, cols, channels)
    print("image size:", img_size)
#     print("image original:", img)
#     print("image reshape:", img_to_1D)
#     return img_to_1D

In [ ]:
# get_img_info(get_image(3)) 

## Preprocessing the  Data

We need to separate the labeled data from the unlabeled

Also need to encode labels as numbers instead of strings

In [ ]:
#is_unlabelled = labels_df["Train Label"] == NAN
labelled = (labels_df["Train Label"]!= "NAN")
unlabelled = (labels_df["Train Label"] == "NAN")

unlabelled_data = (labels_df[unlabelled])
# print(unlabelled_data)

labelled_data = (labels_df[labelled])
# print(labelled_data)

In [ ]:
labeled = []
unlabeled = []

labeled = labelled_data["Train Label"].tolist()
unlabeled = unlabelled_data["Train Label"].tolist()

# print (labeled) 
# print (unlabeled) 

In [ ]:
#col = labels_df.columns
#for i, entry in enumerate(labels_df[col[0]]):
 #   if (entry == 'NAN'):
  #      unlabeled.append(i)
   # else:
   #     labeled.append([i, entry])
# print("labeled", labeled)
# print("unlabeled", unlabeled)


In [ ]:
# Need to reshape training data and labels
train_images, train_labels, test_images, test_labels = [], [], [], []
for i in labelled_data.iterrows():
    train_images.append(get_image(i[0]))
    train_labels.append(i[1]['Train Label'])
print(len(train_images))
for i in range(36):
    plt.subplot(6,6,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
    plt.grid(False)
    plt.imshow(train_images[i])
    plt.xlabel(train_labels[i])
plt.show()

In [ ]:
# print(train_labels)
print(len(train_labels))

One-Hot Encoding

##### USED FOLLOWING TUTORIAL: https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/#:~:text=A%20one%20hot%20encoding%20allows,output%20variables%20that%20are%20categorical.

In [ ]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

##One hot 
values = array(train_labels)
print(values)

In [ ]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values) 
print(integer_encoded)

In [ ]:
onehot_encoder = OneHotEncoder(sparse =False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
print(len(onehot_encoded))

In [ ]:
#change this to one-hot
# for i in range(len(train_labels)):
#     for j in range(len(labels)):
#         if train_labels[i] == labels[j]:
#             train_labels[i] = j
# print (train_labels)


for i in range(len(train_labels)):
    train_labels[i] = onehot_encoded[i]
    
print(len(train_labels))

In [ ]:
#split labeled data so that we have some labeled testing data 80/20
test_images =  np.array(train_images[1600:])
train_images = np.array(train_images[:1600])
test_labels = np.array(train_labels[1600:])
train_labels = np.array(train_labels[:1600])
print(len(train_images), len(train_labels), len(test_images), len(test_labels))

# print(train_images, test_images)


## Baseline Model

### Convolutional Neural Network

As input, a CNN takes tensors of shape (height, width, channels) = (32, 32, 3)

This is based on https://www.tensorflow.org/tutorials/images/cnn

In [ ]:
# model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same',input_shape=(32, 32, 3)))
# model.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform',padding='same'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
# model.add(layers.Conv2D(64, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(128, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
# model.add(layers.Conv2D(128, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Flatten())
# model.add(layers.Dense(128, activation='relu', kernel_initializer='he_uniform'))
# model.add(layers.Dense(10, activation='softmax'))
# model.summary()

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
# model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

Maybe try Stochastic Gradient Descent optimizer?

In [ ]:
callback1 = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
callback2 = tf.keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0, patience=5, restore_best_weights=True)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy']) 

In [ ]:
history = model.fit(train_images, train_labels, epochs=100, 
                    validation_data=(test_images, test_labels), callbacks=[callback1])

## Augmenting the Model



### Label some of our unlabeled data

In [ ]:
new_labels = [];
new_images = [];

for j,i in enumerate(unlabelled_data.iterrows()):
    if j < 500:
        new_images.append(get_image(i[0]))
#         print(i[0])


predictions = model.predict(np.array(new_images))
for i in predictions:
    new_labels.append(get_label(i))
    
print(len(new_images), len(new_labels))

In [ ]:
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
    plt.grid(False)
    plt.imshow(new_images[i])
    plt.xlabel(new_labels[i])
plt.show()

## TODO: one-hot encode predicted labels and concatenate with the original labels

In [ ]:
new_train_images = np.concatenate((train_images, new_images), axis=0)
new_train_labels = np.concatenate((train_labels, new_labels), axis=0)

for i in range(len(new_train_labels)):
    for j in range(len(labels)):
        if new_train_labels[i] == labels[j]:
            new_train_labels[i] = j
            
new_train_labels = tf.strings.to_number(new_train_labels)
            
print(len(new_train_images),len(new_train_labels))

print(new_train_labels, train_labels)


## Training model with new data

In [ ]:
new_model = models.Sequential()
new_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
new_model.add(layers.MaxPooling2D((2, 2)))
new_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
new_model.add(layers.MaxPooling2D((2, 2)))
new_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
new_model.add(layers.Flatten())
new_model.add(layers.Dense(64, activation='relu'))
new_model.add(layers.Dense(10))
new_model.summary()

new_model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy']) 

history2 = new_model.fit(new_train_images, new_train_labels, epochs=100, 
                    validation_data=(test_images, test_labels), callbacks=[callback1])

## ResNet 50 

As per TA recommendation, try a much more complex model

## TODO: figure this out

In [ ]:
# resnet = ResNet50(weights="imagenet", include_top=False, input_shape=(128,128,3), pooling='max')
resnet = ResNet50(include_top=False, input_shape=(128,128,3), pooling='max')

for layer in resnet.layers:
#     print(layer)
    layer.trainable = False

    
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
normal = tf.keras.layers.BatchNormalization()
dropout =tf.keras.layers.Dropout(0.4)
output = tf.keras.layers.Dense(10, activation='softmax')(resnet.output)


In [ ]:
res = tf.keras.models.Model(resnet.input, output)
res.summary()

In [ ]:
resnet_model = tf.keras.models.Sequential()
resnet_model.add(tf.keras.layers.Conv2DTranspose(3, (3, 3), strides=2, padding='same', activation='relu', input_shape=(32,32,3)))
resnet_model.add(tf.keras.layers.BatchNormalization())
resnet_model.add(tf.keras.layers.Conv2DTranspose(3, (3, 3), strides=2, padding='same', activation='relu'))
resnet_model.add(tf.keras.layers.BatchNormalization())
resnet_model.add(res)

In [ ]:
resnet_model.summary()

In [ ]:
resnet_model.compile(optimizer=Adam(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = resnet_model.fit(train_images, train_labels, epochs=20, batch_size=32, verbose=1,
                    validation_data=(test_images, test_labels), callbacks=[callback1])

## TODO: Predict on some labels with resnet model, one-hot-encode those, concatenate with training data, then retrain with model below. 

In [ ]:
resnet_model2 = tf.keras.models.Sequential()
resnet_model2.add(tf.keras.layers.Conv2DTranspose(3, (3, 3), strides=2, padding='same', activation='relu', input_shape=(32,32,3)))
resnet_model2.add(tf.keras.layers.BatchNormalization())
resnet_model2.add(tf.keras.layers.Conv2DTranspose(3, (3, 3), strides=2, padding='same', activation='relu'))
resnet_model2.add(tf.keras.layers.BatchNormalization())
resnet_model2.add(res)

resnet_model2.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = resnet_model2.fit(new_train_images, new_train_labels, epochs=100, 
                    validation_data=(test_images, test_labels), callbacks=[callback1])

# Predictions

Using our model and ResNet


## Write predictions to csv



In [ ]:
import csv

f = open("submission.csv","w+")

pass